In [ ]:
import pandas as pd
import numpy as np
import openai
import json
import robin_stocks as rs
from dataScrape import scooper, collector

In [ ]:
with open('../../GPT_KEY.json') as f:
    data = json.load(f)
openai.api_key = data["key"]

In [ ]:
with open('../../rh_log.json') as f:
    data = json.load(f)
username = data["username"]
pw = data["pw"]

rs.login(username, pw)

In [ ]:
from gpt import GPT
from gpt import Example

In [ ]:
gpt = GPT(engine="davinci",
          temperature=0.5,
          max_tokens=10
)

In [ ]:
with open('trading_news.json') as f:
    data = json.load(f)

In [ ]:
for i in range(len(data)):
    inpt = data['{}'.format(i)][0]
    outpt = data['{}'.format(i)][1]
    gpt.add_example(Example('{}'.format(inpt), '{}'.format(outpt)))

In [ ]:
test = "Grubhub Generates Sales, Not Profit, During Pandemic"

In [ ]:
gpt.submit_request(test)

In [ ]:
with open('sector_matching.json') as f:
    data = json.load(f)

prompt = data["prompt"]

In [ ]:
prompt

In [ ]:
rising_sym = []
falling_sym = []

for stk in rs.markets.get_top_movers('up'):
    scooper(stk, rising_sym)

for stk in rs.markets.get_top_movers('down'):
    scooper(stk, falling_sym)

pooled_sym = rising_sym + falling_sym


In [ ]:
rs.stocks.get_name_by_symbol(pooled_sym[4][0])

In [ ]:
gpt = GPT(engine="davinci",
          temperature=0.5,
          max_tokens=8
)

In [ ]:
from collections import defaultdict

sector_match = defaultdict(str)

for p in pooled_sym:
    req = gpt.submit_request(prompt.format(rs.stocks.get_name_by_symbol(p[0])))
    text = req['choices'][0]['text']
    text = text.split('\n')[0]
    text = text.split(': ')[1]
    sector_match[p] = text